In [ ]:
# Importing all the required libraries

import numpy as np
import array as arr
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import wave
import pylab
from pathlib import Path
from scipy import signal
from scipy.io import wavfile
from sklearn.metrics import confusion_matrix
import itertools
import random
from tensorflow.keras import regularizers
import librosa
import librosa.display

In [ ]:
# Set paths to input and output data

INPUT_DIR = r'E:\00 PS-1\Project\imp\Training audio samples'
OUTPUT_DIR = r'E:\00 PS-1\Project\imp'
# OUTPUT_DIR --> Where you want to save the spectrogram images.

In [ ]:
# Print names of 5 WAV files from the input path

parent_list = os.listdir(INPUT_DIR)
for i in range(5):
    print(parent_list[i])

In [ ]:
# Plot first 3 WAV files as a waveform and a frequency spectrum

for i in range(3): 
    signal_wave = wave.open(os.path.join(INPUT_DIR, parent_list[i]), 'r')
    sample_rate = 16000
    sig = np.frombuffer(signal_wave.readframes(sample_rate), dtype=np.int16)

    plt.figure(figsize=(12,12))
    plot_a = plt.subplot(211)
    plot_a.set_title(parent_list[i])
    plot_a.plot(sig)
    plot_a.set_xlabel('sample rate * time')
    plot_a.set_ylabel('energy')

    plot_b = plt.subplot(212)
    plot_b.specgram(sig, NFFT=1024, Fs=sample_rate, noverlap=900)
    plot_b.set_xlabel('Time')
    plot_b.set_ylabel('Frequency')

plt.show()

In [ ]:
# Plotting Spectrograms for the audio samples and saving them


# Utility function to get sound and frame rate info
np.seterr(divide = 'ignore') 
def get_wav_info(wav_file):
    wav = wave.open(wav_file, 'r')
    frames = wav.readframes(-1)
    sound_info = pylab.frombuffer(frames, 'int16')
    frame_rate = wav.getframerate()
    wav.close()
    return sound_info, frame_rate

# For every recording, make a spectogram and save it as label_class_no.png
if not os.path.exists(os.path.join(OUTPUT_DIR, 'audio-images')):
    os.mkdir(os.path.join(OUTPUT_DIR, 'audio-images'))
    
for filename in os.listdir(INPUT_DIR):
    if "wav" in filename:
        file_path = os.path.join(INPUT_DIR, filename)
        file_stem = Path(file_path).stem
        target_dir = f'class_{file_stem[:2]}'
        dist_dir = os.path.join(os.path.join(OUTPUT_DIR, 'audio-images'), target_dir)
        file_dist_path = os.path.join(dist_dir, file_stem)
        if not os.path.exists(file_dist_path + '.png'):
            if not os.path.exists(dist_dir):
                os.mkdir(dist_dir)
            file_stem = Path(file_path).stem
            sound_info, frame_rate = get_wav_info(file_path)
            pylab.specgram(sound_info, Fs=frame_rate)
            pylab.savefig(f'{file_dist_path}.png')
            pylab.close()

# Print the ten classes in our dataset
path_list = os.listdir(os.path.join(OUTPUT_DIR, 'audio-images'))
print("Classes: \n")
for i in range(16):
    print(path_list[i])
    
# File names for class 1
path_list = os.listdir(os.path.join(OUTPUT_DIR, 'audio-images/class_1_'))
print("\nA few example files: \n")
for i in range(16):
    print(path_list[i])

In [ ]:
# Declare constants
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
BATCH_SIZE = 32
N_CHANNELS = 3
N_CLASSES = 16

# Make a dataset containing the training spectrograms
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=BATCH_SIZE,
                                             validation_split=0.2,
                                             directory=os.path.join(OUTPUT_DIR, 'audio-images'),
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                             subset="training",
                                             seed=0)

# Make a dataset containing the validation spectrogram
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=BATCH_SIZE,
                                             validation_split=0.2,
                                             directory=os.path.join(OUTPUT_DIR, 'audio-images'),
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                             subset="validation",
                                             seed=0)

In [ ]:
plt.figure(figsize=(12, 12))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")
plt.show()

In [ ]:
# Function to prepare our datasets for modelling + Data Augmentation

def prepare(ds, augment=False):
    # Define our one transformation
    rescale = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.Rescaling(1./255)])
    flip_and_rotate = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
    ])
    
    # Apply rescale to both datasets and augmentation only to training
    ds = ds.map(lambda x, y: (rescale(x, training=True), y))
    if augment: ds = ds.map(lambda x, y: (flip_and_rotate(x, training=True), y))
    return ds

train_dataset = prepare(train_dataset, augment=False)
valid_dataset = prepare(valid_dataset, augment=False)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
# Create CNN model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
#model.add(tf.keras.layers.Conv2D(32, 3, strides=1, padding='same', activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
#model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3, strides=2, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128, 3, strides=2, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())

# REGULARIZATION
# Add Dropout layer
model.add(tf.keras.layers.Dropout(0.2))
# Add L2 regularization to the dense layers
model.add(tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))

# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(), metrics=['accuracy'])

# Train model for 100 epochs, capture the history
from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=5, restore_best_weights=True)
history = model.fit(train_dataset, epochs=100, validation_data=valid_dataset)

In [ ]:
# Plot the loss curves for training and validation.

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values)+1)

plt.figure(figsize=(8,6))
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plot the accuracy curves for training and validation.

acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(acc_values)+1)

plt.figure(figsize=(8,6))
plt.plot(epochs, acc_values, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Compute the final loss and accuracy

final_loss, final_acc = model.evaluate(valid_dataset, verbose=0)
print("Final loss: {0:.3f}, final accuracy: {1:.3f}".format(final_loss, final_acc))

In [ ]:
# Load the test dataset
TST_DIR = (r'E:\00 PS-1\Project\imp\Test Audio')
TES_DIR = (r'E:\00 PS-1\Project\imp')

np.seterr(divide = 'ignore') 
def get_wav_info(wav_file):
    wav = wave.open(wav_file, 'r')
    frames = wav.readframes(-1)
    sound_info = pylab.frombuffer(frames, 'int16')
    frame_rate = wav.getframerate()
    wav.close()
    return sound_info, frame_rate

# For every recording, make a spectogram and save it as label_class_no.png
if not os.path.exists(os.path.join(TES_DIR, 'Test Dataset')):
    os.mkdir(os.path.join(TES_DIR, 'Test Dataset'))
    
for filename in os.listdir(TST_DIR):
    if "wav" in filename:
        file_path = os.path.join(TST_DIR, filename)
        file_stem = Path(file_path).stem
        target_dir = f'class_{file_stem[:2]}'
        dist_dir = os.path.join(os.path.join(TES_DIR, 'Test Dataset'), target_dir)
        file_dist_path = os.path.join(dist_dir, file_stem)
        if not os.path.exists(file_dist_path + '.png'):
            if not os.path.exists(dist_dir):
                os.mkdir(dist_dir)
            file_stem = Path(file_path).stem
            sound_info, frame_rate = get_wav_info(file_path)
            pylab.specgram(sound_info, Fs=frame_rate)
            pylab.savefig(f'{file_dist_path}.png')
            pylab.close()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)  # Apply necessary preprocessing
TEST_DIR = (r'E:\00 PS-1\Project\imp\Test Dataset')
image_height = IMAGE_HEIGHT
image_width = IMAGE_WIDTH
test_dataset = test_datagen.flow_from_directory(
    directory=TEST_DIR,
    target_size=(image_width, image_height),
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=False
)

# Make predictions
predictions = model.predict(test_dataset)

# Convert predictions to class labels (if needed)
predicted_labels = np.argmax(predictions, axis=1)

# Get the class labels of the test dataset
class_labels = test_dataset.class_indices

# Invert the class labels dictionary
inv_class_labels = {v: k for k, v in class_labels.items()}

# Convert predicted labels to class names
predicted_classes = [inv_class_labels[label] for label in predicted_labels]

# Print the predictions
for i in range(len(predicted_classes)):
    print(f"Image {i+1}: {predicted_classes[i]}")

In [ ]:
import tkinter as tk
from tkinter import filedialog

def browse_file():
    global sample_dir  # Declare the global variable to be modified inside the function
    sample_dir = filedialog.askopenfilename()
    if sample_dir:
        print("Selected File:", sample_dir)

def create_gui():
    root = tk.Tk()
    root.title("File Explorer GUI")
    root.geometry("300x100")

    # Create a button to open the file explorer
    browse_button = tk.Button(root, text="Open File Explorer", command=browse_file)
    browse_button.pack(pady=10)

    # Create a button to close the popup
    close_button = tk.Button(root, text="Close", command = root.destroy)
    close_button.pack(pady=10)

    root.mainloop()

if __name__ == "__main__":
    create_gui()

print('sample_dir = ',sample_dir)

In [ ]:
# Testing for one particular sample

from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)  # Apply necessary preprocessing
TEST_DIR = (r'E:\00 PS-1\Project\imp\Test Dataset')
image_height = IMAGE_HEIGHT
image_width = IMAGE_WIDTH
test_dataset = test_datagen.flow_from_directory(
    directory=TEST_DIR,
    target_size=(image_width, image_height),
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=False
)

# Path to that sample (Copy path and paste here if required)
#sample_image_path = (r'F:\00 PS-1\Project\imp\Test Dataset\class_1\GP 250.png')

# Load the sample image
sample_image_path = sample_dir
sample_image = tf.keras.preprocessing.image.load_img(sample_image_path, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
sample_image_array = tf.keras.preprocessing.image.img_to_array(sample_image)
sample_image_array = np.expand_dims(sample_image_array, axis=0)
sample_image_array = sample_image_array / 255.0  # Normalize the image

# Make prediction for the sample image
prediction = model.predict(sample_image_array)
predicted_label = np.argmax(prediction)

# Get the class labels of the test dataset
class_labels = test_dataset.class_indices

# Invert the class labels dictionary
inv_class_labels = {v: k for k, v in class_labels.items()}

# Convert predicted label to class name
predicted_class = inv_class_labels[predicted_label]

# Print the prediction for the sample image
print(f"Sample Image: {sample_image_path}")
print(f"Prediction: {predicted_class}")

In [ ]:
from matplotlib import style
import tkinter as tk
from tkinter import ttk
from tkinter import *
from PIL import Image, ImageTk

LARGE_FONT= ("Helvetica", 20)
NORM_FONT= ("Helvetica", 12)
SMALL_FONT= ("Helvetica", 10)

if predicted_class == "class_0_":
   output = "Single Man or Crwaling Men at 100m"
   image_path = Image.open(r'F:\00 PS-1\Project\imp\popup images\0_Single Man.gif')

if predicted_class == "class_1_":
   output = "Group of Men at 100m"
   image_path = Image.open(r'F:\00 PS-1\Project\imp\popup images\1_Group of Men.gif')

if predicted_class == "class_2_":
   output = "Group of Men at 150m"
   image_path = Image.open(r'F:\00 PS-1\Project\imp\popup images\1_Group of Men.gif')

if predicted_class == "class_3_":
   output = "Group of Men at 200m"
   image_path = Image.open(r'F:\00 PS-1\Project\imp\popup images\1_Group of Men.gif')

if predicted_class == "class_4_":
   output = "Group of Men at 250m"
   image_path = Image.open(r'F:\00 PS-1\Project\imp\popup images\1_Group of Men.gif')

if predicted_class == "class_5_":
   output = "Group of Men at 300m"
   image_path = Image.open(r'F:\00 PS-1\Project\imp\popup images\1_Group of Men.gif')

if predicted_class == "class_6_":
   output = "Heavy Vehicle at 150m"
   image_path = Image.open(r'F:\00 PS-1\Project\imp\popup images\3_Heavy Vehicle.gif')

if predicted_class == "class_7_":
   output = "Heavy Vehicle at 200m"
   image_path = Image.open(r'F:\00 PS-1\Project\imp\popup images\3_Heavy Vehicle.gif')

if predicted_class == "class_8_":
   output = "Heavy Vehicle at 250m"
   image_path = Image.open(r'F:\00 PS-1\Project\imp\popup images\3_Heavy Vehicle.gif')

if predicted_class == "class_9_":
   output = "Heavy Vehicle at 300m"
   image_path = Image.open(r'F:\00 PS-1\Project\imp\popup images\3_Heavy Vehicle.gif')

if predicted_class == "class_10":
   output = "Light Vehicle at 100m"
   image_path = Image.open(r'F:\00 PS-1\Project\imp\popup images\2_Light Vehicle.gif')

if predicted_class == "class_11":
   output = "Light Vehicle at 200m"
   image_path = Image.open(r'F:\00 PS-1\Project\imp\popup images\2_Light Vehicle.gif')

if predicted_class == "class_12":
   output = "Light Vehicle at 300m"
   image_path = Image.open(r'F:\00 PS-1\Project\imp\popup images\2_Light Vehicle.gif')

if predicted_class == "class_13":
   output = "Tank at 200m"
   image_path = Image.open(r'F:\00 PS-1\Project\imp\popup images\4_Tank.gif')

if predicted_class == "class_14":
   output = "Tank at 400m"
   image_path = Image.open(r'F:\00 PS-1\Project\imp\popup images\4_Tank.gif')

if predicted_class == "class_15":
   output = "Tank at 500m"
   image_path = Image.open(r'F:\00 PS-1\Project\imp\popup images\4_Tank.gif')

popup = tk.Tk()
popup.title("Target Info")
popup.geometry("400x400")

output_label = tk.Label(popup, text=output, font=NORM_FONT)
output_label.pack()

# Create a photoimage object of the image in the path
image1 = Image.open(image_path)
image1 = image1.resize((350, 350), Image.LANCZOS)
test = ImageTk.PhotoImage(image1)
label1 = tk.Label(popup, image=test)
label1.image = test
label1.pack(pady=10)
    
popup.mainloop()

In [ ]:
# wait

In [ ]:
# Clean the output dir (the saved spectrogram images)
# Don't do this unless you are changing the training dataset

import shutil
shutil.rmtree('/kaggle/working/audio-images')

In [ ]:
# Some extra pieces of code that we were experimenting:

In [ ]:
import splitfolders
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio((r'E:\00 PS-1\Project\imp\audio-images/'), output=(r'E:\00 PS-1\Project\imp'), seed=1337, ratio=(.8, .2)) # default values

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255, # rescale all pixel values from 0-255, so after this step all our pixel values are in range (0,1)
        shear_range=0.2, #to apply some random tranfromations
        zoom_range=0.2, #to apply zoom
        horizontal_flip=True) # image will be flipper horiztest_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(
        rescale=1./255, # rescale all pixel values from 0-255, so aftre this step all our pixel values are in range (0,1)
        shear_range=0.2, #to apply some random tranfromations
        zoom_range=0.2, #to apply zoom
        horizontal_flip=True) # image will be flipper horiztest_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Declare constants

IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
BATCH_SIZE = 32
N_CHANNELS = 3
N_CLASSES = 16

# Make a dataset containing the training spectrograms
train_dataset = train_datagen.flow_from_directory(
        (r'E:\00 PS-1\Project\imp\imp Datasets\train'),
        target_size=(256, 256),
        batch_size=32,
        class_mode='sparse',
        shuffle = False)

# Make a dataset containing the validation spectrogram
valid_dataset = test_datagen.flow_from_directory(
        (r'E:\00 PS-1\Project\imp\imp Datasets\val'),
        target_size=(256, 256),
        batch_size=32,
        class_mode='sparse',
        shuffle = False )

In [ ]:
# Creating an array train_labels

TR_DIR = (r'E:\00 PS-1\Project\imp\imp Datasets\train/')
tr_labels = []
for filename in os.listdir(TR_DIR):
    if "wav" in filename:
        file_path = os.path.join(TR_DIR, filename)
        file_stem = Path(file_path).stem
        tr_labels.append(file_stem[:2])
print(tr_labels)

train_labels = np.array(tr_labels)
print(train_labels)

In [ ]:
# DATA AUGMENTATION

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of the ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Generate augmented data
augmented_train_dataset = datagen.flow(train_dataset, train_labels, batch_size=BATCH_SIZE)

In [ ]:
# HYPERPARAMETER TUNING

from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Define a function to create the CNN model
def create_model(optimizer='adam', dropout_rate=0.2):
    model = tf.keras.models.Sequential()
    # Define your model architecture here
    # ...
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create the KerasClassifier wrapper
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the hyperparameters and their values to search
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'dropout_rate': [0.2, 0.3, 0.4]
}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(train_dataset, train_labels)

# Print the best hyperparameters and corresponding accuracy
print("Best Hyperparameters: ", grid_result.best_params_)
print("Best Accuracy: ", grid_result.best_score_)

In [ ]:
# HYPERPARAMETER TUNING 2

from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Define a function to create the CNN model
def create_model(optimizer='adam', dropout_rate=0.2):
    model = tf.keras.models.Sequential()
    # Define your model architecture here
    # ...
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create the KerasClassifier wrapper
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the hyperparameters and their values to search
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'dropout_rate': [0.2, 0.3, 0.4]
}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(train_dataset)

# Print the best hyperparameters and corresponding accuracy
print("Best Hyperparameters: ", grid_result.best_params_)
print("Best Accuracy: ", grid_result.best_score_)

In [ ]:
# LEARNING RATE

from tensorflow.keras.callbacks import LearningRateScheduler

# Define a learning rate schedule function

def lr_schedule(epoch):
    initial_learning_rate = 0.01
    decay = 0.1
    lr = initial_learning_rate * decay ** (epoch // 10)
    return lr

# Use the learning rate schedule during training

lr_scheduler = LearningRateScheduler(lr_schedule)
model.fit(train_dataset, train_labels, epochs=100, callbacks=[lr_scheduler])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the image data generator for training data
train_datagen = ImageDataGenerator(rescale=1./255)

# Set the path to your train dataset directory
#train_directory = train_dataset

# Set the target image size and batch size
image_width, image_height = 256, 256
batch_size = 32

# Extract the train labels
train_labels = train_dataset.classes

# Print the train labels
print(train_labels)

In [ ]:
# define test set and training set
test_set = valid_dataset
training_set = train_dataset

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the parameters for data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

image_width = 256
image_height = 256

# Generate the augmented training set
train_generator = datagen.flow_from_directory(
    r'E:\00 PS-1\Project\imp\audio-images',  # Path to the training data directory
    target_size=(image_width, image_height),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

# Access the class indices
class_indices = train_generator.class_indices
print(class_indices)

In [ ]:
import numba
wav_file=(r'E:\00 PS-1\Project\Samples -30 June 2023\New folder\DOPPLER COLLECTION\GP 2 MEN CRAWLING 200M\0 DEGREE APPROCH\200A0.wav')
#preprocess the audio file
wav =  librosa.load(wav_file, res_type='kaiser_fast')

# sound_info, frame_rate = get_wav_info(file_path)
specgram_features = pylab.specgram(wav)
specgram_scaled_features = np.mean(specgram_features.T,axis=0)
#Reshape MFCC feature to 2-D array
specgram_scaled_features=specgram_scaled_features.reshape(1,-1)
predicted_label=model.predict_classes(specgram_features)
x_predict=model.predict(specgram_scaled_features) 
predicted_label=np.argmax(x_predict,axis=1)
print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
print(prediction_class)

In [ ]:
# test_set.reset()
pred = model.predict_generator(test_set, steps=50, verbose=1)

In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)

labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predictions = predictions[:200]
filenames=test_set.filenames

In [ ]:
print(len(filename, len(predictions)))
# (200, 200)

In [ ]:
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions},orient='index')
results.to_csv("prediction_results.csv",index=False)